# Module 1 Homework: Docker & SQL

In this homework we'll prepare the environment and practice
Docker and SQL

## Question 1. Understanding Docker images

Run docker with the `python:3.13` image. Use an entrypoint `bash` to interact with the container.

What's the version of `pip` in the image?

```bash
docker run -it --entrypoint bash python:3.13
pip -V
```

## Question 2. Understanding Docker networking and docker-compose

Given the following `docker-compose.yaml`, what is the `hostname` and `port` that pgadmin should use to connect to the postgres database?

```yaml
services:
  db:
    container_name: postgres
    image: postgres:17-alpine
    environment:
      POSTGRES_USER: 'postgres'
      POSTGRES_PASSWORD: 'postgres'
      POSTGRES_DB: 'ny_taxi'
    ports:
      - '5433:5432'
    volumes:
      - vol-pgdata:/var/lib/postgresql/data

  pgadmin:
    container_name: pgadmin
    image: dpage/pgadmin4:latest
    environment:
      PGADMIN_DEFAULT_EMAIL: "pgadmin@pgadmin.com"
      PGADMIN_DEFAULT_PASSWORD: "pgadmin"
    ports:
      - "8080:80"
    volumes:
      - vol-pgadmin_data:/var/lib/pgadmin

volumes:
  vol-pgdata:
    name: vol-pgdata
  vol-pgadmin_data:
    name: vol-pgadmin_data
```

Answer：```db:5432```

## Question 3. Counting short trips

For the trips in November 2025 (lpep_pickup_datetime between '2025-11-01' and '2025-12-01', exclusive of the upper bound), how many trips had a `trip_distance` of less than or equal to 1 mile?

In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("green_tripdata_2025-11.parquet")
df

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,7.20,...,0.5,1.94,0.0,NaN,1.0,11.64,1.0,1.0,0.00,0.00
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,7.20,...,0.5,0.00,0.0,NaN,1.0,9.70,2.0,1.0,0.00,0.00
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,13.50,...,0.5,5.00,0.0,NaN,1.0,21.00,1.0,1.0,0.00,0.00
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,24.70,...,0.5,0.50,0.0,NaN,1.0,27.70,1.0,1.0,0.00,0.00
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,18.40,...,1.5,1.00,0.0,NaN,1.0,24.65,1.0,1.0,2.75,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46907,2,2025-11-30 19:58:34,2025-11-30 20:14:28,None,NaN,59,51,NaN,8.50,33.22,...,0.5,0.00,0.0,NaN,1.0,34.72,NaN,NaN,NaN,0.00
46908,2,2025-11-30 19:34:00,2025-11-30 19:46:00,None,NaN,74,151,NaN,1.73,13.86,...,0.5,0.77,0.0,NaN,1.0,16.13,NaN,NaN,NaN,0.00
46909,2,2025-11-30 21:46:46,2025-11-30 22:17:55,None,NaN,33,163,NaN,7.52,38.42,...,0.5,1.00,0.0,NaN,1.0,44.42,NaN,NaN,NaN,0.75
46910,2,2025-11-30 21:00:00,2025-11-30 21:15:00,None,NaN,16,95,NaN,5.61,24.67,...,0.5,0.00,0.0,NaN,1.0,26.17,NaN,NaN,NaN,0.00


In [3]:
date_mask = (df['lpep_pickup_datetime'] >= '2025-11-01') & (df['lpep_pickup_datetime'] < '2025-12-01')
distance_mask = df['trip_distance'] <= 1.0
count = len(df[date_mask & distance_mask])
print(f"There were {count} trips that had a `trip_distance` of less than or equal to 1 mile.")

There were 8007 trips that had a `trip_distance` of less than or equal to 1 mile.


## Question 4. Longest trip for each day

Which was the pick up day with the longest trip distance? Only consider trips with `trip_distance` less than 100 miles (to exclude data errors).

Use the pick up time for your calculations.

In [4]:
df['pickup_date'] = df['lpep_pickup_datetime'].dt.date
filtered_df = df[df['trip_distance'] < 100].copy()
daily_max_distances = filtered_df.groupby('pickup_date')['trip_distance'].max()
longest_trip_day = daily_max_distances.idxmax()

print(f"The day with the longest trip was: {longest_trip_day}")

The day with the longest trip was: 2025-11-14


## Question 5. Biggest pickup zone

Which was the pickup zone with the largest `total_amount` (sum of all trips) on November 18th, 2025?

In [5]:
amount = df[df['pickup_date'] == pd.to_datetime("2025-11-18").date()].groupby('PULocationID')['total_amount'].sum()
df_zones = pd.read_csv("taxi_zone_lookup.csv")
zone = df_zones.loc[amount.idxmax(), "Zone"]
print(f"The zone with the highest total amount on 2025-11-18 was: {zone}")

The zone with the highest total amount on 2025-11-18 was: East Harlem South


## Question 6. Largest tip

For the passengers picked up in the zone named "East Harlem North" in November 2025, which was the drop off zone that had the largest tip?

Note: it's `tip` , not `trip`. We need the name of the zone, not the ID.

In [6]:
filtered_df = df[df["PULocationID"] == df_zones.loc[df_zones["Zone"] == "East Harlem North", "LocationID"].values[0]]
zone = df_zones.loc[df_zones["LocationID"] == filtered_df.loc[filtered_df["tip_amount"].idxmax(), "DOLocationID"], "Zone"].values[0]
print(f"The dropoff zone for the trip with the highest tip in East Harlem North was: {zone}")

The dropoff zone for the trip with the highest tip in East Harlem North was: Yorkville West


## Terraform

In this section homework we'll prepare the environment by creating resources in GCP with Terraform.

In your VM on GCP/Laptop/GitHub Codespace install Terraform.
Copy the files from the course repo
[here](../../../01-docker-terraform/terraform/terraform) to your VM/Laptop/GitHub Codespace.

Modify the files as necessary to create a GCP Bucket and Big Query Dataset.

## Question 7. Terraform Workflow

Which of the following sequences, respectively, describes the workflow for:
1. Downloading the provider plugins and setting up backend,
2. Generating proposed changes and auto-executing the plan
3. Remove all resources managed by terraform`

```bash
terraform init
terraform apply -auto-approve
terraform destroy
```

## Submitting the solutions

* Form for submitting: https://courses.datatalks.club/de-zoomcamp-2026/homework/hw1